In [1]:
import pandas as pd
import os, MySQLdb
import numpy as np
import missingno as msno
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
%config InlineBackend.figure_formats = ['svg']

### We extracted all statistics_name that pertains to 'All Heroes', we ignored hero-specific statistics, for better uniformity.

In [2]:
r = []
for i in [2018, 2019, 2020]:
    basedir = os.path.join(os.getcwd(), f'phs_{i}', '')
    for f in os.listdir(basedir):
        df = pd.read_csv(basedir + f)
        if 'hero' in df.columns.tolist():
            tmp = df[df['hero'] == 'All Heroes']['stat_name']
        elif 'hero_name' in df.columns.tolist():
            tmp = df[df['hero_name'] == 'All Heroes']['stat_name']

        r = list(set(r + tmp.to_list()))

r = set(r)
print(len(r))

36


### Construct An Empty Table

In [3]:
table = pd.DataFrame(columns=sorted(r))
table.columns

Index(['All Damage Done', 'Assists', 'Average Time Alive',
       'Barrier Damage Done', 'Damage - Quick Melee', 'Damage Blocked',
       'Damage Done', 'Damage Taken', 'Deaths', 'Defensive Assists',
       'Eliminations', 'Environmental Deaths', 'Environmental Kills',
       'Final Blows', 'Healing Done', 'Hero Damage Done', 'Knockback Kills',
       'Melee Final Blows', 'Melee Percentage of Final Blows', 'Multikills',
       'Objective Kills', 'Objective Time', 'Offensive Assists',
       'Recon Assists', 'Shots Fired', 'Solo Kills',
       'Teleporter Pads Destroyed', 'Time Alive', 'Time Building Ultimate',
       'Time Elapsed per Ultimate Earned', 'Time Holding Ultimate',
       'Time Played', 'Turrets Destroyed', 'Ultimates Earned - Fractional',
       'Ultimates Used', 'Weapon Accuracy'],
      dtype='object')

### Fill in table entries (traversing through players again)

In [4]:
PLAYER = []
for i in [2018, 2019, 2020]:
    basedir = os.path.join(os.getcwd(), f'phs_{i}', '')
    for f in sorted(list(os.listdir(basedir))):
        df = pd.read_csv(basedir + f)
        col = df.columns.tolist()
        for j in df.itertuples():
            if 'player' in col:
                rown = f'{j.player.lower()}{i}'
            elif 'player_name' in col:
                rown = f'{j.player_name.lower()}{i}'
            
            try:
                if str(j.hero).lower() == 'all heroes':
                    table.loc[rown, j.stat_name] = j.stat_amount
                    PLAYER.append(rown)
            except:
                if str(j.hero_name).lower() == 'all heroes':
                    table.loc[rown, j.stat_name] = j.stat_amount
                    PLAYER.append(rown)

        print(f"{f}\t\t{table.shape}")

PLAYER = set(PLAYER)

phs_2018_playoffs.csv		(44, 36)
phs_2018_stage_1.csv		(105, 36)
phs_2018_stage_2.csv		(112, 36)
phs_2018_stage_3.csv		(123, 36)
phs_2018_stage_4.csv		(129, 36)
phs_2019_playoffs.csv		(212, 36)
phs_2019_stage_1.csv		(297, 36)
phs_2019_stage_2.csv		(312, 36)
phs_2019_stage_3.csv		(331, 36)
phs_2019_stage_4.csv		(336, 36)
phs_2020_1.csv		(542, 36)
phs_2020_2.csv		(549, 36)


### Extract relevant player name (all in lower case)

In [5]:
extract = sorted(set([i[:-4] for i in PLAYER]))
extract

['ado',
 'adora',
 'agilities',
 'aid',
 'aimgod',
 'akm',
 'alarm',
 'alemao',
 'altering',
 'although',
 'ameng',
 'anamo',
 'ans',
 'apply',
 'architect',
 'arhan',
 'ark',
 'asher',
 'ating',
 'axxiom',
 'babel',
 'babybay',
 'baconjack',
 'bani',
 'bazzi',
 'bdosin',
 'beast',
 'bebe',
 'benbest',
 'bernar',
 'bianca',
 'bigg00se',
 'birdring',
 'bischu',
 'blase',
 'blasé',
 'boink',
 'boombox',
 'bqb',
 'brussen',
 'bumper',
 'bunny',
 'byrem',
 'carcar',
 'carpe',
 'chara',
 'chipsa',
 'chipshajen',
 'choihyobin',
 'clestyn',
 'clockwork',
 'closer',
 'cocco',
 'coldest',
 'colourhex',
 'coma',
 'coolmatt',
 'corey',
 'cr0ng',
 'creative',
 'crimzo',
 'custa',
 'cwoosh',
 'daco',
 'daemin',
 'dafran',
 'dalton',
 'danteh',
 'danye',
 'dayfly',
 'dding',
 'decay',
 'dhak',
 'diem',
 'diya',
 'dogman',
 'doha',
 'dpi',
 'dreamer',
 'dreamkazper',
 'edison',
 'effect',
 'eileen',
 'elk',
 'ellivote',
 'elsa',
 'envy',
 'eqo',
 'erster',
 'fahzix',
 'fate',
 'fctfctn',
 'fdgod',
 '

### Show the full table

In [6]:
table

,All Damage Done,Assists,Average Time Alive,Barrier Damage Done,Damage - Quick Melee,Damage Blocked,Damage Done,Damage Taken,Deaths,Defensive Assists,...,Teleporter Pads Destroyed,Time Alive,Time Building Ultimate,Time Elapsed per Ultimate Earned,Time Holding Ultimate,Time Played,Turrets Destroyed,Ultimates Earned - Fractional,Ultimates Used,Weapon Accuracy
gamsu2018,11018.6,25,122.151,1962.79,455.86,11279.5,9055.81,12565.7,6,4,...,NaN,675.878,625.556,101.119,135.755,803.059,NaN,6.18632,5,0.548611
kellex2018,6654.15,9,55.1209,2834.15,60,NaN,3795,4426.78,6,21,...,NaN,675.876,596.574,93.1689,99.221,735.858,NaN,6.40314,4,0.261084
mistakes2018,17412.8,14,89.7347,5058.2,92.4971,803.716,12106,5693.03,5,6,...,NaN,668.003,663.436,74.1882,112.053,803.059,NaN,8.94261,6,0.386288
neko2018,14863,15,133.274,4083.15,260.641,56,10776.8,5450.3,5,29,...,NaN,686.058,599.779,83.4366,165.67,803.059,NaN,7.18844,6,0.344029
note2018,18346.8,14,85.7364,6691.16,72.9168,8057.53,11406.2,13052.4,4,3,...,NaN,685.891,580.801,103.303,220.682,803.059,NaN,5.62229,21,0.349774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ta1yo2020,7800.42,11,148.417,2816.69,159,NaN,4983.73,2373.83,2,NaN,...,NaN,296.834,277.616,91.0928,4.216,296.859,NaN,3.04762,2,0.222477
bianca2020,8795.79,7,110.698,893.649,30,16020,6502.24,13303.2,7,2,...,NaN,715.983,653.146,63.858,125.446,780.667,NaN,10.2281,8,0.194922
lr1s2020,4055.07,5,76.9573,811.52,30,NaN,3243.55,2609.69,3,6,...,NaN,307.829,237.515,50.9873,68.934,336.807,NaN,4.65831,4,0.111111
saucy2020,8106.02,10,110.966,2981.3,60,5759.41,5109.72,2755.66,4,NaN,...,NaN,665.795,522.026,110.989,158.902,695.897,4,4.70339,3,0.404231


### Removing columns that have too many missing values

In [33]:
prop = 0.1
tokeep = ~(table.isnull().sum() > prop * table.shape[0])
r1_table = table[tokeep.iloc[tokeep.values].index]
print(f"Removing columns with >{prop} missingness\nLeft with {r1_table.shape[1]} statistics on performance")
print("*" * 30)
print(f"Current Missingness")
print(r1_table.isnull().sum() / table.shape[0] * 100)

Removing columns with >0.1 missingness
Left with 28 statistics on performance
******************************
Current Missingness
All Damage Done                     0.182149
Assists                             0.182149
Average Time Alive                  0.728597
Barrier Damage Done                 0.182149
Damage - Quick Melee                1.457195
Damage Done                         3.278689
Damage Taken                        3.096539
Deaths                              0.364299
Eliminations                        0.182149
Environmental Deaths                8.014572
Final Blows                         0.182149
Hero Damage Done                    0.182149
Melee Final Blows                   6.193078
Melee Percentage of Final Blows     6.193078
Multikills                          5.282332
Objective Kills                     0.364299
Objective Time                      0.000000
Offensive Assists                   2.914390
Shots Fired                         0.910747
Solo Kills      

### Set the missing values to be the average of the value in its column

In [8]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
r2_table = imp_mean.fit_transform(r1_table)
r2_table = pd.DataFrame(r2_table, columns=r1_table.columns, index=r1_table.index)
# Check to see that all values are indeed not NaN nor NULL
assert (r2_table.isnull().sum().values == 0).all()
r2_table

,All Damage Done,Assists,Average Time Alive,Barrier Damage Done,Damage - Quick Melee,Damage Done,Damage Taken,Deaths,Eliminations,Environmental Deaths,...,Shots Fired,Solo Kills,Time Alive,Time Building Ultimate,Time Elapsed per Ultimate Earned,Time Holding Ultimate,Time Played,Ultimates Earned - Fractional,Ultimates Used,Weapon Accuracy
gamsu2018,11018.600720,25.0,122.150879,1962.792554,455.859944,9055.808162,12565.747800,6.0,30.0,1.000000,...,810.0,1.0,675.878021,625.556018,101.119279,135.755004,803.058990,6.186318,5.0,0.548611
kellex2018,6654.146255,9.0,55.120891,2834.146255,60.000000,3795.000000,4426.775113,6.0,9.0,1.000000,...,809.0,1.0,675.876020,596.574017,93.168926,99.221003,735.858478,6.403144,4.0,0.261084
mistakes2018,17412.776710,14.0,89.734669,5058.199062,92.497137,12105.959560,5693.025097,5.0,24.0,1.000000,...,1412.0,1.0,668.003020,663.436019,74.188223,112.053003,803.058990,8.942606,6.0,0.386288
neko2018,14862.974460,15.0,133.274004,4083.153479,260.641460,10776.820980,5450.304399,5.0,23.0,1.000000,...,713.0,2.0,686.058021,599.779017,83.436590,165.670005,803.058990,7.188441,6.0,0.344029
note2018,18346.830110,14.0,85.736378,6691.161931,72.916759,11406.206500,13052.415760,4.0,28.0,1.000000,...,20039.0,2.0,685.891020,580.801017,103.303350,220.682006,803.058990,5.622286,21.0,0.349774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ta1yo2020,7800.417480,11.0,148.417004,2816.689651,159.000000,4983.727828,2373.825726,2.0,16.0,1.184158,...,529.0,1.0,296.834009,277.616008,91.092753,4.216000,296.858802,3.047619,2.0,0.222477
bianca2020,8795.790688,7.0,110.697753,893.648767,30.000000,6502.239303,13303.183119,7.0,13.0,1.000000,...,11132.0,1.0,715.983022,653.146019,63.858027,125.446004,780.667209,10.228096,8.0,0.194922
lr1s2020,4055.066810,5.0,76.957252,811.519937,30.000000,3243.546873,2609.690491,3.0,6.0,1.000000,...,10.0,1.0,307.829009,237.515006,50.987335,68.934002,336.807380,4.658314,4.0,0.111111
saucy2020,8106.018867,10.0,110.965837,2981.296954,60.000000,5109.721913,2755.660169,4.0,13.0,1.000000,...,2293.0,2.0,665.795020,522.026016,110.989273,158.902005,695.896706,4.703392,3.0,0.404231


### As we read on the Internet, scaling the values to be between 0 and 1 are helpful

In [9]:
scaler = MinMaxScaler()
r3_table = scaler.fit_transform(r2_table)
r3_table = pd.DataFrame(r3_table, columns=r1_table.columns, index=r1_table.index)
r3_table.columns

Index(['All Damage Done', 'Assists', 'Average Time Alive',
       'Barrier Damage Done', 'Damage - Quick Melee', 'Damage Done',
       'Damage Taken', 'Deaths', 'Eliminations', 'Environmental Deaths',
       'Final Blows', 'Hero Damage Done', 'Melee Final Blows',
       'Melee Percentage of Final Blows', 'Multikills', 'Objective Kills',
       'Objective Time', 'Offensive Assists', 'Shots Fired', 'Solo Kills',
       'Time Alive', 'Time Building Ultimate',
       'Time Elapsed per Ultimate Earned', 'Time Holding Ultimate',
       'Time Played', 'Ultimates Earned - Fractional', 'Ultimates Used',
       'Weapon Accuracy'],
      dtype='object')

### We removed "shots fired" based on Jingguo's player domain specific knowledge

In [10]:
r3_table = r3_table.drop(['Shots Fired'], axis=1)
r3_table.shape

(549, 27)

### Compute Team Accuracy and Player Accuracy for each year

***accuracy*** a map between interested element and corresponding accuracy

***lib*** a map details the team that players are on, in a specific year

***record*** team's winning rate in specific years

In [11]:
accuracy = dict(zip(r3_table.index, [-1 for i in range(r3_table.shape[0])]))

In [12]:
player_hist = pd.read_csv('player_history.csv', index_col=0)
players = set(player_hist['Player Name'].values.tolist())
lib = {}
for i in players:
    for j in range(2018, 2021):
        lib[f"{i.lower()}{j}"] = []

# Obtain which team that the player served in (in 2018, 2019, and 2020)
for p in player_hist.itertuples():
    try:
        sy = int(p.start_time.split('-')[0]) if p.start_time != 'Present' else 2020
        ey = int(  p.end_time.split('-')[0]) if p.end_time   != 'Present' else 2020
        if sy <= 2018 and ey >= 2018:
            lib[f"{p._1.lower()}2018"].append(p.Team)
        if sy <= 2019 and ey >= 2019:
            lib[f"{p._1.lower()}2019"].append(p.Team)
        if sy <= 2020 and ey >= 2020:
            lib[f"{p._1.lower()}2020"].append(p.Team)
        if ey < 2018:
            continue
        
    except:
        print(p)


Pandas(Index=48, _1='Apply', start_time='201?-??-??', end_time='2017-12-18', Team='Sinatraas rap playlist')
Pandas(Index=160, _1='Carcar', start_time='201?-??-??', end_time='2017-12-18', Team='Sinatraas rap playlist')
Pandas(Index=176, _1='Chipshajen', start_time='2015-??-??', end_time='201?-??-??', Team='HULKTastic')
Pandas(Index=672, _1='KSF', start_time='201?-??-??', end_time='2017-01-??', Team='EnVision eSports')
Pandas(Index=752, _1='Kyo', start_time='????-??-??', end_time='2016-08-31', Team='Oversalt')
Pandas(Index=791, _1='LiNkzr', start_time='2015-11-16', end_time='201?-??-??', Team=' DEALWITHPEPE')
Pandas(Index=1016, _1='Paintbrush', start_time='201?-??-??', end_time='2018-02-??', Team='No Clout')
Pandas(Index=1111, _1='SADO', start_time='2020-11-21', end_time=' Present', Team=' Toronto Defiant')
Pandas(Index=1206, _1='Snow_(Brazilian_player)', start_time='201?-??-??', end_time='201?-??-??', Team='Lightning Storm')
Pandas(Index=1207, _1='Snow_(Brazilian_player)', start_time='2

#### See descriptions above for comments

In [13]:
df = pd.read_csv("match_map_stats.csv")
for i in range(2018, 2021):
    # Extract the list of participating teams
    line = df['round_end_time'].str.contains(str(i), na=-1, regex=False)
    l = df.iloc[np.where(line.values)[0]]
    teams = l[['map_winner', 'map_loser']].values.flatten()
    teams = set(teams)
    teams.remove('draw')
    record = dict(zip(teams, [np.array([0.0, 0.0]) for i in range(len(teams))]))

    # We used the winning rate formula described in https://en.wikipedia.org/wiki/Winning_percentage
    for j in l[['map_winner', 'map_loser', 'team_one_name', 'team_two_name']].values:
        if j[0].lower() != 'draw':
            record[j[0]] += [1, 1]
            record[j[1]] += [0, 1]
        else:
            record[j[2]] += [0.5, 1]
            record[j[3]] += [0.5, 1]
    
    # Sanity Check Total Pts should be half of total number of games
    l = np.array([0.0, 0.0])
    for m, n in record.items():
        l += n
    assert l[0] * 2 == l[1]

    # For all players in list
    for p in extract:
        entry = f"{p}{i}"
        # If we are interested in this player's performance in this year
        if entry in accuracy:
            # If we know the teams of player+year combination
            if entry in lib:
                t = lib[entry]
                score, count = 0.0, 0.0
                for l in t:
                    if l.strip() in record:
                        score += record[l.strip()][0] / record[l.strip()][1]  
                        count += 1
                # If we can find at least winning rate 
                # of one team that player played on
                if count != 0:
                    accuracy[entry] = score / count
                else:
                    print(f"{entry}\t{t}")
            else:
                print(entry)

    print(set(teams))


snow2018
{'Shanghai Dragons', 'London Spitfire', 'Los Angeles Gladiators', 'Philadelphia Fusion', 'Boston Uprising', 'Florida Mayhem', 'Houston Outlaws', 'Seoul Dynasty', 'New York Excelsior', 'Los Angeles Valiant', 'San Francisco Shock', 'Dallas Fuel'}
kyo2019	['XTEN Esports', 'T1me']
{'Paris Eternal', 'Boston Uprising', 'Chengdu Hunters', 'Washington Justice', 'Vancouver Titans', 'San Francisco Shock', 'New York Excelsior', 'Toronto Defiant', 'Shanghai Dragons', 'London Spitfire', 'Los Angeles Gladiators', 'Philadelphia Fusion', 'Atlanta Reign', 'Houston Outlaws', 'Florida Mayhem', 'Seoul Dynasty', 'Los Angeles Valiant', 'Guangzhou Charge', 'Hangzhou Spark', 'Dallas Fuel'}
beast2020	['Dignity']
blase2020
kyo2020	[]
{'Team Custa', 'Jehong and Students', 'Paris Eternal', 'Boston Uprising', 'Washington Justice', 'Chengdu Hunters', 'Vancouver Titans', 'Brick Movers', 'Triple A', 'San Francisco Shock', 'New York Excelsior', 'Team Jake', 'Team Reinhardt', 'Toronto Defiant', 'Shanghai Drago

In [14]:
# We can concatenate the winning rate list with our original table
acc = pd.DataFrame.from_dict(accuracy, orient='index',
                       columns=['accuracy'])
r4 = pd.concat([r3_table, acc], axis=1)
# Removing the player+year combination that corresponds to -1 winning rate
r5 = r4.iloc[(np.where((r4['accuracy'] != -1) == True)[0])]

In [15]:
# We extract X and Y
X = r5.values[:,:-1]
y = r5.values[:, -1]

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, LinearRegression, SGDRegressor
from sklearn import svm, tree
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error

### K Fold cross-validation (tried several models, as listed on Sklearn API page)

In [17]:

num_model, num_split = 8, 10

kf = KFold(n_splits=num_split, shuffle=True)
ev = np.zeros(num_model)
mae = np.zeros(num_model)
mse = np.zeros(num_model)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = Ridge(alpha=1).fit(X_train, y_train)
    ev[0] += explained_variance_score(y_test, clf.predict(X_test))
    mae[0] += mean_absolute_error(y_test, clf.predict(X_test))
    mse[0] += mean_squared_error(y_test, clf.predict(X_test))
    
    clf = LinearRegression().fit(X_train, y_train)
    ev[1] += explained_variance_score(y_test, clf.predict(X_test))
    mae[1] += mean_absolute_error(y_test, clf.predict(X_test))
    mse[1] += mean_squared_error(y_test, clf.predict(X_test))

    clf = svm.SVR().fit(X_train, y_train)
    ev[2] += explained_variance_score(y_test, clf.predict(X_test))
    mae[2] += mean_absolute_error(y_test, clf.predict(X_test))
    mse[2] += mean_squared_error(y_test, clf.predict(X_test))
    
    clf = SGDRegressor(max_iter=1e7, tol=1e-10).fit(X_train, y_train)
    ev[3] += explained_variance_score(y_test, clf.predict(X_test))
    mae[3] += mean_absolute_error(y_test, clf.predict(X_test))
    mse[3] += mean_squared_error(y_test, clf.predict(X_test))

    clf = tree.DecisionTreeRegressor().fit(X_train, y_train)
    ev[4] += explained_variance_score(y_test, clf.predict(X_test))
    mae[4] += mean_absolute_error(y_test, clf.predict(X_test))
    mse[4] += mean_squared_error(y_test, clf.predict(X_test))
    
    clf = RandomForestRegressor(max_depth=5, min_samples_split=4).fit(X_train, y_train)
    ev[5] += explained_variance_score(y_test, clf.predict(X_test))
    mae[5] += mean_absolute_error(y_test, clf.predict(X_test))
    mse[5] += mean_squared_error(y_test, clf.predict(X_test))

    clf = ExtraTreesRegressor(max_depth=5, min_samples_split=4).fit(X_train, y_train)
    ev[6] += explained_variance_score(y_test, clf.predict(X_test))
    mae[6] += mean_absolute_error(y_test, clf.predict(X_test))
    mse[6] += mean_squared_error(y_test, clf.predict(X_test))

    clf = AdaBoostRegressor(n_estimators=100, learning_rate=0.1, loss='exponential').fit(X_train, y_train)
    ev[7] += explained_variance_score(y_test, clf.predict(X_test))
    mae[7] += mean_absolute_error(y_test, clf.predict(X_test))
    mse[7] += mean_squared_error(y_test, clf.predict(X_test))    
    
    
print(ev/num_split)
print(mae/num_split)
print(mse/num_split)

[ 0.06839186  0.04561581 -0.01100386  0.04974854 -1.07050017  0.08890856
  0.07592068  0.06497723]
[0.09325806 0.09434309 0.09683268 0.09361203 0.13113221 0.09220809
 0.09137934 0.09358398]
[0.01455566 0.01492926 0.01540849 0.01476975 0.03111739 0.01405997
 0.01438025 0.01446851]


##### Based on experimentation, we see that RandomForest Regressor can consistently outperform other ML models (it has the highest explained_variance_score). To generate winning rate for all players, we will fit a RandomForest model on the entire dataset and predict on all of it. 

In [18]:
kf = KFold(n_splits=num_split, shuffle=True, random_state=10)
clf = RandomForestRegressor(max_depth=5, min_samples_split=4).fit(X, y)
m = clf.predict(X)

In [24]:
prediction = dict(zip(r5.index, m))

In [25]:
final = {}
for p in extract:
    a, c = 0, 0
    for y in range(2018, 2021):
        entry = f"{p}{y}"
        if entry in prediction:
            a += prediction[entry]
            c += 1
    if c != 0:
        final[p] = a/c
            

In [26]:
final = {k: v for k, v in sorted(final.items(), key=lambda item: item[1], reverse=True)}

### We will connect to the database and feed in our data

In [28]:
database = MySQLdb.connect(host="dbase.cs.jhu.edu", user="20fa_rzhai2", passwd="AgadM8or", db="20fa_rzhai2_db")
cursor = database.cursor()
query = "INSERT INTO Predict(PlayerName, WinningRate) VALUES (%s, %s)"

In [29]:
def rebuild_Predict_table():
    cursor.execute("DROP TABLE Predict")
    definition = (f"CREATE TABLE Predict(PlayerName VARCHAR(50), WinningRate FLOAT(40, 20), PRIMARY KEY (PlayerName))")
    cursor.execute(definition)
    print("Re-built")
    
rebuild_Predict_table()

Re-built


In [30]:
def view_table(name):
    cursor.execute(f"SELECT * FROM {name}")
    myresult = cursor.fetchall()
    for x in myresult:
        print(x)

In [31]:
cursor.execute("TRUNCATE TABLE Predict")
for k, v in final.items():
    cursor.execute(query, (k, v))
    
print(f"{len(final)} entries inserted")
view_table("Predict")
database.commit()
cursor.close()
database.close()

316 entries inserted
('ado', 0.4528316557407379)
('adora', 0.4897456765174866)
('agilities', 0.505233883857727)
('aid', 0.39446529746055603)
('aimgod', 0.4945133328437805)
('akm', 0.48682740330696106)
('alarm', 0.507260262966156)
('alemao', 0.47814804315567017)
('altering', 0.21590730547904968)
('although', 0.4806620180606842)
('ameng', 0.47937363386154175)
('anamo', 0.46220341324806213)
('ans', 0.5010608434677124)
('apply', 0.47366708517074585)
('architect', 0.5123159885406494)
('arhan', 0.4579475224018097)
('ark', 0.5086613893508911)
('asher', 0.47659701108932495)
('ating', 0.4903634488582611)
('axxiom', 0.47115588188171387)
('babel', 0.4408295452594757)
('babybay', 0.4878408908843994)
('baconjack', 0.4716787040233612)
('bani', 0.4645763337612152)
('bazzi', 0.4768657982349396)
('bdosin', 0.48149386048316956)
('bebe', 0.4985174536705017)
('benbest', 0.4614100158214569)
('bernar', 0.4712281823158264)
('bianca', 0.4776862561702728)
('bigg00se', 0.46552756428718567)
('birdring', 0.490380